In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
# from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pandas as pd
from config import api_key

In [2]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.FollowTheMoney
members_collection = db.members
receipt_totals_collection = db.receiptTotals
candidate_data_collection = db.candidate_data
candidate_data_collection.remove()
members_collection.remove()
receipt_totals_collection.remove()


<ipython-input-2-ab10ed2b4962>:7: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  candidate_data_collection.remove()
<ipython-input-2-ab10ed2b4962>:8: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  members_collection.remove()
<ipython-input-2-ab10ed2b4962>:9: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  receipt_totals_collection.remove()


{'n': 1146, 'ok': 1.0}

In [3]:
newurl=f"https://api.open.fec.gov/v1/schedules/schedule_a/by_state/by_candidate/?sort_null_only=false&api_key={api_key}&sort_hide_null=false&election_full=false&candidate_id=H8KS03155&candidate_id=H0KS02188&candidate_id=%20H8KS04112&candidate_id=H0KS01123&candidate_id=S0KS00315&candidate_id=S0KS00091&candidate_id=S8MO00160&candidate_id=S0MO00183&candidate_id=H8M001143&candidate_id=H2M002102&candidate_id=H8M009153&candidate_id=H0M004086&candidate_id=H4M005234&candidate_id=H0M006073&candidate_id=H0M007113&candidate_id=H4M008162&cycle=2014&cycle=2016&cycle=2018&cycle=2020&per_page=20&page=1&sort_nulls_last=false"
money=requests.get(newurl).json()

In [4]:
total_pages = money['pagination']['pages']
for i in range(1,total_pages+1):
    totals_url=f"https://api.open.fec.gov/v1/schedules/schedule_a/by_state/by_candidate/?sort_null_only=false&api_key={api_key}&sort_hide_null=false&election_full=false&candidate_id=H8KS03155&candidate_id=H0KS02188&candidate_id=20H8KS04112&candidate_id=H0KS01123&candidate_id=S0KS00315&candidate_id=S0KS00091&candidate_id=S8MO00160&candidate_id=S0MO00183&candidate_id=H8M001143&candidate_id=H2M002102&candidate_id=H8M009153&candidate_id=H0M004086&candidate_id=H4M005234&candidate_id=H0M006073&candidate_id=H0M007113&candidate_id=H4M008162&cycle=2014&cycle=2016&cycle=2018&cycle=2020&per_page=20&page={i}&sort_nulls_last=false"
    totals_json = requests.get(totals_url).json()
    for item in totals_json['results']:
        receipt_totals_collection.insert_one(item)

In [5]:
url=f"https://api.open.fec.gov/v1/candidates/?api_key={api_key}"
members = requests.get(f"{url}&election_year=2013&election_year=2014&election_year=2015&election_year=2016&election_year=2017&election_year=2018&election_year=2019&election_year=2020&sort=name&candidate_status=C&page=1&is_active_candidate=true&per_page=20&state=KS&state=MO&office=H&office=S&sort_hide_null=false&sort_null_only=false&sort_nulls_last=false").json()
#find out how many pages of data there are
total_pages = members['pagination']['pages']


In [6]:
#loop through and get all results
for i in range(1,total_pages+1):
    members = requests.get(f"{url}&election_year=2013&election_year=2014&election_year=2015&election_year=2016&election_year=2017&election_year=2018&election_year=2019&election_year=2020&sort=name&candidate_status=C&page={i}&is_active_candidate=true&per_page=20&state=KS&state=MO&office=H&office=S&sort_hide_null=false&sort_null_only=false&sort_nulls_last=false").json()
    #loop through each record and push to DB
    for item in members["results"]:
        members_collection.insert_one(item)

In [7]:
# grab the kansas canidates from tbe DB
KS_candidates = members_collection.find({"state":"KS"})
MO_candidates = members_collection.find({"state":"MO"})


In [8]:
#populat kansas
for cand in KS_candidates:
    candDetail_url = f"https://api.open.fec.gov/v1/candidate/{cand['candidate_id']}/filings/?sort=-receipt_date&sort_nulls_last=false&page=1&sort_hide_null=false&per_page=20&api_key=cKUzXnqasaxmD3l9mDOOD5wqRfZ0rnYuZJzxKnhx&sort_null_only=false"
    c_response = requests.get(candDetail_url).json()
    if c_response['results']:
        candidate_data_collection.insert_many(c_response['results'])

In [9]:
candidates = candidate_data_collection.find()
for candidate in candidates:
    print(candidate)

{'_id': ObjectId('60acfa4661c0b68d89f6dc26'), 'sub_id': '4112720171468625137', 'previous_file_number': 1192456, 'report_year': 2017, 'coverage_start_date': None, 'senate_personal_funds': None, 'additional_bank_names': None, 'bank_depository_street_1': None, 'candidate_id': 'H8KS02207', 'election_year': 2018, 'report_type_full': None, 'amendment_chain': [1192456.0], 'csv_url': 'https://docquery.fec.gov/csv/456/1192456.csv', 'fec_file_id': 'FEC-1192456', 'form_type': 'F2', 'primary_general_indicator': None, 'cycle': 2018, 'form_category': 'STATEMENT', 'state': 'KS', 'fec_url': 'https://docquery.fec.gov/dcdev/posted/1192456.fec', 'update_date': '2017-11-27T00:00:00', 'committee_type': None, 'party': 'REP', 'amendment_version': 0, 'committee_name': None, 'pages': 1, 'total_disbursements': None, 'office': 'H', 'cash_on_hand_end_period': None, 'most_recent': True, 'document_type': None, 'total_communication_cost': None, 'document_description': 'Statement of candidacy 2017', 'coverage_end_dat

In [11]:
#populat missouri
for cand in MO_candidates:
    candDetail_url = f"https://api.open.fec.gov/v1/candidate/{cand['candidate_id']}/filings/?sort=-receipt_date&sort_nulls_last=false&page=1&sort_hide_null=false&per_page=20&api_key=cKUzXnqasaxmD3l9mDOOD5wqRfZ0rnYuZJzxKnhx&sort_null_only=false"
    c_response = requests.get(candDetail_url).json()
    if c_response['results']:
        candidate_data_collection.insert_many(c_response['results'])

In [13]:
candidates2 = candidate_data_collection.find()
for candidate in candidates2:
    print(candidate)

{'_id': ObjectId('60acfa4661c0b68d89f6dc26'), 'sub_id': '4112720171468625137', 'previous_file_number': 1192456, 'report_year': 2017, 'coverage_start_date': None, 'senate_personal_funds': None, 'additional_bank_names': None, 'bank_depository_street_1': None, 'candidate_id': 'H8KS02207', 'election_year': 2018, 'report_type_full': None, 'amendment_chain': [1192456.0], 'csv_url': 'https://docquery.fec.gov/csv/456/1192456.csv', 'fec_file_id': 'FEC-1192456', 'form_type': 'F2', 'primary_general_indicator': None, 'cycle': 2018, 'form_category': 'STATEMENT', 'state': 'KS', 'fec_url': 'https://docquery.fec.gov/dcdev/posted/1192456.fec', 'update_date': '2017-11-27T00:00:00', 'committee_type': None, 'party': 'REP', 'amendment_version': 0, 'committee_name': None, 'pages': 1, 'total_disbursements': None, 'office': 'H', 'cash_on_hand_end_period': None, 'most_recent': True, 'document_type': None, 'total_communication_cost': None, 'document_description': 'Statement of candidacy 2017', 'coverage_end_dat

In [17]:
from bson import json_util
from flask import Flask, jsonify
import pymongo
import os
import json



In [18]:
app = Flask(__name__)

In [19]:
db = client.FollowTheMoney

In [20]:
@app.route("/")
def default():
    return (os.environ.get('NAME', 'Name not configured'))

In [21]:
@app.route("/api")
def api():
    data = members_collection.find()
    
    return json_util.dumps(data)

In [22]:
port = int(os.environ.get('PORT', 5000))

In [23]:
if __name__ == '__main__':
    app.run(debug=True, port=port)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with fsevents reloader


SystemExit: 1

/Users/angeliquetucker/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
%tb

No traceback available to show.
